In [1]:
import numpy as np
import math
import time
import json
import pandas as pd
import requests
from pandas import json_normalize
import os
import csv

In [2]:
season_id = ['37','38','39','40','41','21','23','24','25','27','2']

def load_file(season_id, getter="remote", path = None):
    resp = requests.get(f"https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/11/{season_id}.json")
    season_dict = json.loads(resp.text)
    season_df = json_normalize(season_dict, sep="_")
    return season_dict, season_df


def load_match_file(match_id, getter="remote", path = None):
    resp = requests.get(f"https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{match_id}.json")
    match_dict = json.loads(resp.text)
    df = json_normalize(match_dict, sep="_")
    return match_dict, df
def get_starters(match_dict, side="home"):
    lineups = match_dict[0]["tactics"]["lineup"] if side == "home" else match_dict[1]["tactics"]["lineup"]
    return lineups
# match_dict, df = load_match_file(69299, getter="remote")
# lineups = get_starters(match_dict, side="home")
# lineups
match_id_list = []
for i in season_id:
    season_dict, season_df = load_file(i, getter="remote")    
    match_id_list.append(season_df.match_id.tolist())  
matchlist = [item for sublist in match_id_list for item in sublist]

In [10]:
r = len(matchlist)
r

311

In [8]:
import sys
import time

def print_percent_done(index, total, bar_len=50, title='Please wait'):
    '''
    index is expected to be 0 based index. 
    0 <= index < total
    '''
    percent_done = (index+1)/total*100
    percent_done = round(percent_done, 1)

    done = round(percent_done/(100/bar_len))
    togo = bar_len-done

    done_str = '█'*int(done)
    togo_str = '░'*int(togo)

    print(f'\t⏳{title}: [{done_str}{togo_str}] {percent_done}% done', end='\r')

    if round(percent_done) == 100:
        print('\t✅')

In [11]:
passes_df = pd.DataFrame()
for i in range(len(matchlist)):
    match_dict, df = load_match_file(matchlist[i], getter="remote")
    passes=df.query("(type_name == 'Pass')&(team_name=='Barcelona')\
                   &(pass_type_name not in ['Free Kick', 'Corner', 'Throw-in', 'Kick Off'])")
    passes_df = passes_df.append(passes)
    print_percent_done(i,r)

	✅Please wait: [██████████████████████████████████████████████████] 99.7% done
	✅Please wait: [██████████████████████████████████████████████████] 100.0% done


In [12]:
len(passes_df)

194504

In [13]:
passes_df.to_csv('Passesfrom11seasons.csv')